In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/UniCon_EnetV2_OpenMax/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
import tensorflow as tf
import numpy as np

# /Users/hieutran/lab/unicon-ids/data/can-ml/preprocessed/all_features/TFRecord_w32_s16/2
tfrecord_file_path = '../data/can-ml/2017-subaru-forester/preprocessed/size_64_10/TFRecord_w64_s32/2/train/0.tfrec'
def parse_tfrecord_fn(example):
    feature_description = {
        'id_seq': tf.io.FixedLenFeature([64, 64], tf.int64),
        'data_seq': tf.io.FixedLenFeature([64, 64], tf.int64),
        'timestamp': tf.io.FixedLenFeature([64, 64], tf.float32),
        'label': tf.io.FixedLenFeature([1], tf.int64)
    }
    return tf.io.parse_single_example(example, feature_description)

# Tạo dataset từ TFRecord file
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_path)
parsed_dataset = raw_dataset.map(parse_tfrecord_fn)

# Hiển thị toàn bộ nội dung của timestamp
for i, parsed_record in enumerate(parsed_dataset.take(5)):  # Lấy 5 bản ghi đầu tiên
    timestamp_array = parsed_record['data_seq'].numpy()  # Chuyển sang NumPy array
    print(f"Record {i+1} - Timestamp data:")
    print(np.array2string(timestamp_array, threshold=np.inf))  # In toàn bộ dữ liệu
    print("\n" + "="*50 + "\n")


Record 1 - Timestamp data:
[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0
  1 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0]
 [0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 1
  0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1]
 [1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1
  1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
  1 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0
  1 1 1 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0]
 [1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 1 0 1 0 

In [9]:
import time
import torch
from networks.efficient_net import ConEfficientNet
# Chọn thiết bị chạy thử nghiệm (GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang chạy trên: {device}")

# Tạo batch giả lập (1 ảnh 64x64)
test_input = torch.randn(1, 3, 64, 64).to(device)

# Khởi tạo mô hình và chuyển sang thiết bị thích hợp
model = ConEfficientNet(embedding_dim=1792, feat_dim=128, head='mlp', pretrained=False).to(device)
model.eval()

# Đo thời gian inference 100 lần để lấy trung bình
num_trials = 100
timing_results = []

with torch.no_grad():
    for _ in range(num_trials):
        torch.cuda.synchronize() if torch.cuda.is_available() else None  # Đồng bộ hóa GPU nếu cần
        start_time = time.time()
        _ = model(test_input)  # Chạy mô hình
        torch.cuda.synchronize() if torch.cuda.is_available() else None  # Đồng bộ hóa GPU nếu cần
        end_time = time.time()
        timing_results.append((end_time - start_time) * 1000)  # Chuyển sang milliseconds

# Tính trung bình thời gian inference
avg_inference_time = sum(timing_results) / num_trials

print(f"Thời gian suy luận trung bình cho một frame 64 messages: {avg_inference_time:.2f} ms")


Đang chạy trên: cuda
Thời gian suy luận trung bình cho một frame 64 messages: 23.33 ms


In [20]:
import time
import torch
from networks.efficient_net_lite import ConEfficientNet
# Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang chạy trên thiết bị: {device}")

# Tạo batch input giả lập
test_input = torch.randn(1, 3, 64, 64).to(device)

# Khởi tạo mô hình EfficientNet-B0 Pruned
model = ConEfficientNet(embedding_dim=1024, feat_dim=128, head='mlp', pretrained=False).to(device)
model.eval()

# Đo thời gian inference 100 lần
num_trials = 100
timing_results = []

with torch.no_grad():
    for _ in range(num_trials):
        torch.cuda.synchronize() if torch.cuda.is_available() else None  # Đồng bộ hóa GPU nếu cần
        start_time = time.time()
        _ = model(test_input)
        torch.cuda.synchronize() if torch.cuda.is_available() else None  # Đồng bộ hóa GPU nếu cần
        end_time = time.time()
        timing_results.append((end_time - start_time) * 1000)  # ms

# Kết quả
avg_inference_time = sum(timing_results) / num_trials
print(f"Thời gian suy luận trung bình (EfficientNet-B0 Pruned): {avg_inference_time:.2f} ms")


Đang chạy trên thiết bị: cuda
Thời gian suy luận trung bình (EfficientNet-B0 Pruned): 6.88 ms


In [4]:
from networks.efficient_net_lite import ConEfficientNet
import torch
# Chọn thiết bị
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Đang chạy trên thiết bị: {device}")
model = ConEfficientNet(embedding_dim=1280, feat_dim=128, head='mlp', pretrained=False).to(device)
test_input = torch.randn(1, 3, 64, 64).to(device)


Đang chạy trên thiết bị: cuda


In [5]:
output = model(test_input)
print("Kích thước đầu ra của mô hình:", output.size())

Kích thước đầu ra của mô hình: torch.Size([1, 128])


In [9]:
# Hàm hook để in ra kích thước đầu vào và đầu ra của từng lớp
import torch.nn as nn
def print_shapes_hook(module, input, output):
    print(f"{module.__class__.__name__.ljust(20)} | "
          f"Input Shape: {str(input[0].shape).ljust(30)} | "
          f"Output Shape: {str(output.shape)}")

for name, module in model.encoder._modules.items():
    # Đối với các MBConv block, bạn sẽ cần đăng ký hook vào các lớp con của nó
    if isinstance(module, nn.Module):
        # Đăng ký hook vào tất cả các lớp con
        for sub_name, sub_module in module.named_children():
            sub_module.register_forward_hook(print_shapes_hook)


# Tiến hành kiểm tra thông qua forward pass
output = model(test_input)

Conv2d               | Input Shape: torch.Size([1, 3, 64, 64])     | Output Shape: torch.Size([1, 32, 64, 64])
BatchNorm2d          | Input Shape: torch.Size([1, 32, 64, 64])    | Output Shape: torch.Size([1, 32, 64, 64])
ReLU6                | Input Shape: torch.Size([1, 32, 64, 64])    | Output Shape: torch.Size([1, 32, 64, 64])
Conv2dStaticSamePadding | Input Shape: torch.Size([1, 320, 4, 4])     | Output Shape: torch.Size([1, 1280, 4, 4])
BatchNorm2d          | Input Shape: torch.Size([1, 1280, 4, 4])    | Output Shape: torch.Size([1, 1280, 4, 4])
ReLU6                | Input Shape: torch.Size([1, 1280, 4, 4])    | Output Shape: torch.Size([1, 1280, 4, 4])
Linear               | Input Shape: torch.Size([1, 1280])          | Output Shape: torch.Size([1, 1280])
Linear               | Input Shape: torch.Size([1, 1280])          | Output Shape: torch.Size([1, 1280])


In [6]:
from open_gan import Generator, Discriminator
G = Generator()
D = Discriminator()
import torch
z = torch.randn(4, 128)


In [7]:
# Forward Generator
gen_feat = G(z)
print(f"[Generator Output] shape: {gen_feat.shape}, norm: {gen_feat.norm(p=2, dim=1)}, min: {gen_feat.min().item():.4f}, max: {gen_feat.max().item():.4f}")


[Generator Output] shape: torch.Size([4, 8256]), norm: tensor([ 7.4615,  9.0911, 10.1059,  9.3146],
       grad_fn=<LinalgVectorNormBackward0>), min: -0.5246, max: 0.4161


In [8]:

score = D(gen_feat)
print(f"[Discriminator Score] shape: {score.shape}, values: {score.view(-1).tolist()}")

[Discriminator Score] shape: torch.Size([4, 1]), values: [0.00765092670917511, 0.01895749941468239, 0.00991226639598608, 0.00663892924785614]
